In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

In [3]:
def fetch_stock_data(ticker, period="1y", interval="1d"):
    return yf.download(ticker, interval=interval, period=period)

In [4]:
def calculate_atr(df,atr_period=14):
    df['High-Low'] = df['High']-df['Low']
    df['High-PrevClose']=np.abs(df['High']-df['Close'].shift(1))
    df['Low-PrevClose'] =np.abs(df['Low']-df['Close'].shift(1))
    
    df['TrueRange']=df[['High-Low','High-PrevClose','Low-PrevClose']].max(axis=1)
    df['ATR']=df['TrueRange'].rolling(window=atr_period).mean()
    
    return df

In [9]:
def calculate_support_resistance(df, atr_multiplier=1):
    
    current_strike_price = df['Close'].iloc[-1]
    latest_atr = df['ATR'].iloc[-1]
    
    percentage_zone=(latest_atr*atr_multiplier)/current_strike_price

    support_level=current_strike_price*(1-percentage_zone)
    resistance_level=current_strike_price*(1+percentage_zone)

    return current_strike_price,latest_atr,percentage_zone,support_level,resistance_level


In [13]:
def main():
    
    ticker=input("Enter the stock ticker symbol (e.g., ^NSEI for NIFTY): ").strip()
    time_frame=input("Enter the time frame for prediction (e.g., 1d, 1h, 15m): ").strip()

    stock_data=fetch_stock_data(ticker,interval=time_frame)
    stock_data = calculate_atr(stock_data)

    current_strike_price,latest_atr,percentage_zone,support_level,resistance_level=calculate_support_resistance(stock_data)

    print(f"\nStock Ticker: {ticker}")
    print(f"Time Frame: {time_frame}")
    print(f"Most Recent Date: {stock_data.index[-1].date()}")
    print(f"Current Strike Price: {current_strike_price:.2f}")
    print(f"Latest ATR: {latest_atr:.2f}")
    print(f"Calculated Percentage Zone: {percentage_zone * 100:.2f}%")
    print(f"Ideal Support Level: {support_level:.2f}")
    print(f"Ideal Resistance Level: {resistance_level:.2f}")


if __name__=="__main__":
    main()

Enter the stock ticker symbol (e.g., ^NSEI for NIFTY): Zomato.ns
Enter the time frame for prediction (e.g., 1d, 1h, 15m): 1h
[*********************100%%**********************]  1 of 1 completed

Stock Ticker: Zomato.ns
Time Frame: 1h
Most Recent Date: 2024-10-25
Current Strike Price: 253.80
Latest ATR: 4.60
Calculated Percentage Zone: 1.81%
Ideal Support Level: 249.20
Ideal Resistance Level: 258.40
